In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
print(application_df.shape)
application_df.head()

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Noisy variables are removed from features

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'], axis = 1) 
print(application_df.shape)
application_df.head(10)

(34299, 9)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1580        1
C1248        1
C2380        1
C4500        1
C1370        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Additional neurons are added to hidden layers

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3900      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 9,001
Trainable params: 9,001
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_1/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_1/weights.{epoch:02d}hdf5"

In [20]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 860us/step - loss: 0.5939 - accuracy: 0.6973
Epoch 2/100
804/804 [==============================] - 1s 876us/step - loss: 0.5482 - accuracy: 0.7356
Epoch 3/100
804/804 [==============================] - 1s 856us/step - loss: 0.5548 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 1s 898us/step - loss: 0.5552 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 1s 907us/step - loss: 0.5472 - accuracy: 0.7375
Epoch 6/100
804/804 [==============================] - 1s 850us/step - loss: 0.5491 - accuracy: 0.7351
Epoch 7/100
804/804 [==============================] - 1s 920us/step - loss: 0.5471 - accuracy: 0.7361
Epoch 8/100
804/804 [==============================] - 1s 925us/step - loss: 0.5458 - accuracy: 0.73870s - loss: 0.5450 - accu
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7344
Epoch 10/100
804/804 [=============================

804/804 [==============================] - 1s 855us/step - loss: 0.5348 - accuracy: 0.7475
Epoch 78/100
804/804 [==============================] - 1s 853us/step - loss: 0.5443 - accuracy: 0.7377
Epoch 79/100
804/804 [==============================] - 1s 837us/step - loss: 0.5401 - accuracy: 0.7400
Epoch 80/100
804/804 [==============================] - 1s 839us/step - loss: 0.5367 - accuracy: 0.7410
Epoch 81/100
804/804 [==============================] - 1s 897us/step - loss: 0.5407 - accuracy: 0.7372
Epoch 82/100
804/804 [==============================] - 1s 832us/step - loss: 0.5424 - accuracy: 0.7376
Epoch 83/100
804/804 [==============================] - 1s 842us/step - loss: 0.5441 - accuracy: 0.7377
Epoch 84/100
804/804 [==============================] - 1s 869us/step - loss: 0.5394 - accuracy: 0.7403
Epoch 85/100
804/804 [==============================] - 1s 837us/step - loss: 0.5360 - accuracy: 0.7417
Epoch 86/100
804/804 [==============================] - 1s 847us/step - loss:

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5640 - accuracy: 0.7234
Loss: 0.5639860033988953, Accuracy: 0.7233819365501404


### Additional hidden layers are added

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3120      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_5 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 21        
Total params: 6,191
Trainable params: 6,191
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_2/",exist_ok=True)
checkpoint_path = "checkpoints_optimization_2/weights.{epoch:02d}hdf5"

In [26]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80,callbacks=[cp_callback])

Epoch 1/80
804/804 [==============================] - 1s 913us/step - loss: 0.5904 - accuracy: 0.7101
Epoch 2/80
804/804 [==============================] - 1s 959us/step - loss: 0.5561 - accuracy: 0.7352
Epoch 3/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7323
Epoch 4/80
804/804 [==============================] - 1s 958us/step - loss: 0.5558 - accuracy: 0.7301
Epoch 5/80
804/804 [==============================] - 1s 894us/step - loss: 0.5485 - accuracy: 0.7352

Epoch 00005: saving model to checkpoints_optimization_2/weights.05hdf5
Epoch 6/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7335
Epoch 7/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7331
Epoch 8/80
804/804 [==============================] - 1s 907us/step - loss: 0.5505 - accuracy: 0.7336
Epoch 9/80
804/804 [==============================] - 1s 915us/step - loss: 0.5463 - accuracy: 0.7363
Epoch 10/80
804/

804/804 [==============================] - 1s 885us/step - loss: 0.5343 - accuracy: 0.7455

Epoch 00070: saving model to checkpoints_optimization_2/weights.70hdf5
Epoch 71/80
804/804 [==============================] - 1s 962us/step - loss: 0.5410 - accuracy: 0.7372
Epoch 72/80
804/804 [==============================] - 1s 952us/step - loss: 0.5401 - accuracy: 0.7399
Epoch 73/80
804/804 [==============================] - 1s 908us/step - loss: 0.5396 - accuracy: 0.7400
Epoch 74/80
804/804 [==============================] - 1s 935us/step - loss: 0.5361 - accuracy: 0.7399
Epoch 75/80
804/804 [==============================] - 1s 972us/step - loss: 0.5366 - accuracy: 0.7425

Epoch 00075: saving model to checkpoints_optimization_2/weights.75hdf5
Epoch 76/80
804/804 [==============================] - 1s 920us/step - loss: 0.5423 - accuracy: 0.7386
Epoch 77/80
804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7380
Epoch 78/80
804/804 [=========================

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7251
Loss: 0.5601142048835754, Accuracy: 0.7251312136650085


### The activation function of hidden layers or output layers is changed for optimization

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                3120      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 31        
Total params: 5,581
Trainable params: 5,581
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_3/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_3/weights.{epoch:02d}hdf5"

In [31]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)

Epoch 1/100
804/804 [==============================] - 1s 983us/step - loss: 0.5878 - accuracy: 0.71020s - loss: 0
Epoch 2/100
804/804 [==============================] - 1s 923us/step - loss: 0.5604 - accuracy: 0.7271
Epoch 3/100
804/804 [==============================] - 1s 844us/step - loss: 0.5588 - accuracy: 0.7276
Epoch 4/100
804/804 [==============================] - 1s 861us/step - loss: 0.5489 - accuracy: 0.73390s - loss: 0.5483 - accuracy: 0.
Epoch 5/100
804/804 [==============================] - 1s 842us/step - loss: 0.5472 - accuracy: 0.7344
Epoch 6/100
804/804 [==============================] - 1s 855us/step - loss: 0.5491 - accuracy: 0.7358
Epoch 7/100
804/804 [==============================] - 1s 852us/step - loss: 0.5539 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 1s 862us/step - loss: 0.5455 - accuracy: 0.7355
Epoch 9/100
804/804 [==============================] - 1s 860us/step - loss: 0.5498 - accuracy: 0.7330
Epoch 10/100
804/804 [=======

804/804 [==============================] - 1s 830us/step - loss: 0.5340 - accuracy: 0.7459
Epoch 78/100
804/804 [==============================] - 1s 815us/step - loss: 0.5376 - accuracy: 0.7447
Epoch 79/100
804/804 [==============================] - 1s 832us/step - loss: 0.5380 - accuracy: 0.7376
Epoch 80/100
804/804 [==============================] - 1s 813us/step - loss: 0.5348 - accuracy: 0.7429
Epoch 81/100
804/804 [==============================] - 1s 830us/step - loss: 0.5327 - accuracy: 0.7437
Epoch 82/100
804/804 [==============================] - 1s 844us/step - loss: 0.5372 - accuracy: 0.7399
Epoch 83/100
804/804 [==============================] - 1s 824us/step - loss: 0.5368 - accuracy: 0.7414
Epoch 84/100
804/804 [==============================] - 1s 842us/step - loss: 0.5346 - accuracy: 0.7431
Epoch 85/100
804/804 [==============================] - 1s 817us/step - loss: 0.5343 - accuracy: 0.7442
Epoch 86/100
804/804 [==============================] - 1s 849us/step - loss:

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7237
Loss: 0.5600712895393372, Accuracy: 0.7237317562103271


In [33]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")